In [44]:
#Import de librerías necesarias
import pyreadr
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
import random
import time
import pandas as pd
from scipy.stats import chi2
from scipy.stats import chi2_contingency
import scipy.stats as stats
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from scipy.stats import mannwhitneyu
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from collections import defaultdict
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from scipy.stats import spearmanr, kendalltau
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.utils.class_weight import compute_class_weight
from patsy import cr
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from tqdm import tqdm
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, ParameterGrid

In [45]:
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [46]:
#Importar los datos de R a Python
datos = pyreadr.read_r('area_desbalance_ionico.RData')

#Para comprobar que se importan correctamente los datos
#print(datos.keys())

#Este archivo tiene tres variables

#datos_anon_D: datos extraídos de los ECGS - edad, sexo, fecha, hora y 
#nombre del fichero. Hay un código (código) que identifica a cada paciente
datos_anon_DF = datos['datos_anon_DF']

#lead_anon_DF
#variables electrocardiográficas correspondiente a los ECGs de la variable anterior
leads_anon_DF = datos['leads_anon_DF']

#pacs_con_ECGs_DF
#variables de los datos del potasio, fecha de la analítica, edad del paciente, sexo
#nivel de K y código del paciente
pacs_con_ECGs_DF = datos['pacs_con_ECGs_DF']

# Normalidad - Hiperpotasemia - Hipopotasemia

In [47]:
#Primer paso: coger solo los valores de K normales y de hiperpotasemia
k_val = pacs_con_ECGs_DF.copy()

limites = [float('-inf'), 3.5, 5.2, float('inf')]
etiquetas = [1, 0, 2]

In [48]:
k_valores = k_val.copy()

k_valores['categoria'] = pd.cut(k_valores['K'], bins=limites, labels=etiquetas, right=False)

#Se reemplazan los sexos por enteros
#1: Hombre
#0: Mujer
k_valores['Sexo'] = k_valores['Sexo'].replace({'M': 1, 'F': 0})

# Eliminación de variables con datos vacíos

In [49]:
#En leads_anon_DF, se van a eliminar aquellas columnas en las que falte más del 1% de los datos

#El como máximo (el 100%) de datos que podría tener cada característica, sería igual al número total de filas que
#hay registradas
#Se calcula el 10% y se redondea sin decimales
diez_pct_datos = round(len(leads_anon_DF) * 0.01, 0)

#Ahora, se va a coger un subconjunto del dataframe en el que se eliminan las
#características cuyo número de datos sea menor al 10%

#Se cuenta el número de NaN o celdas sin datos en cada columna
celdas_sin_datos = leads_anon_DF.isnull().sum()

#Se cogen las características cuyo número de datos vacíos es mayor o igual al 10%
caracteristicas_eliminar = celdas_sin_datos[celdas_sin_datos >= diez_pct_datos].index

#Se crear el subDataFrame eliminando las columnas correspondientes
leads_anon_DF_limpio = leads_anon_DF.drop(columns = caracteristicas_eliminar)

In [50]:
#Ahora hay que asociar a cada análisis de sangre, su ECG más cercano
def ECG_mas_reciente(fecha, codigo):
     
    archivos_ECGs = datos_anon_DF.loc[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]    
    
    fechas_archivos = archivos_ECGs['date']
    horas_archivos = archivos_ECGs['time']
        
    diferencia_temporal = [abs(fecha - fecha_archivo) for fecha_archivo in fechas_archivos]
    
    ecg = (datos_anon_DF.index[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]).tolist()
    
    if(len(diferencia_temporal) > 0):
    
        combinado = list(zip(diferencia_temporal, ecg))
        combinado = sorted(combinado, key=lambda x: x[0])

        diferencia_temporal, ecg = zip(*combinado)
        
    return ecg, diferencia_temporal

In [51]:
#Ahora, se añade la columna "Categoria" y "Sexo" a leads_anon_DF_limpio
#Se crean esas columnas en el nuevo dataframe con valores a Nan
leads_anon_DF_limpio['Sexo'] = float('NaN')
leads_anon_DF_limpio['categoria'] = float('NaN')
leads_anon_DF_limpio['K'] = float('NaN')
leads_anon_DF_limpio['Edad'] = float('NaN')
ecgs_utilizadas = {} 

for i in range(len(k_valores)):
    fecha_hora = (k_valores['Fecha'].iloc[i]).to_pydatetime()
    fecha = fecha_hora.date()
    
    indice, dist_temporal = ECG_mas_reciente(fecha, k_valores['codigo'].iloc[i])
  
    for j in range(len(indice)):
        #Si el ECGs no tiene una analítica asignada, se le pone
        if indice[j] not in ecgs_utilizadas:
            #Se actualizan los valores de las columnas "Sexo" y "Categoría"
            #de la fila correspondiente
            leads_anon_DF_limpio.at[indice[j], 'Sexo'] = k_valores['Sexo'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'Edad'] = k_valores['Edad'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]

            ecgs_utilizadas[indice[j]] = [k_valores['categoria'].iloc[i], dist_temporal[j].days]

        #Si el ECG ya tiene una analítica asociada, si la nueva analítica tiene un valor
        #de categoría diferente y se asocia con una anormalidad, se cambia
        else:
            if((dist_temporal[j].days < ecgs_utilizadas[indice[j]][1])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days
                
            if((dist_temporal[j].days == ecgs_utilizadas[indice[j]][1]) & (k_valores['categoria'].iloc[i] > ecgs_utilizadas[indice[j]][0])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days                
            
#Se eliminan las filas que tengan la variable objetivo "categoria" a NaN
leads_anon_DF_limpio = leads_anon_DF_limpio.dropna(subset=['categoria'])

In [52]:
pparea = leads_anon_DF_limpio.filter(regex='_pparea', axis=1)
tparea = leads_anon_DF_limpio.filter(regex='_tparea', axis=1)

ppamp = leads_anon_DF_limpio.filter(regex='_ppamp', axis=1)
rpamp = leads_anon_DF_limpio.filter(regex='_rpamp', axis=1)
spamp = leads_anon_DF_limpio.filter(regex='_spamp', axis=1)
tpamp = leads_anon_DF_limpio.filter(regex='_tpamp', axis=1)

rpdur = leads_anon_DF_limpio.filter(regex='_rpdur', axis=1)
spdur = leads_anon_DF_limpio.filter(regex='_spdur', axis=1)
tpdur = leads_anon_DF_limpio.filter(regex='_tpdur', axis=1)

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=ppamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpdur.columns)

In [53]:
parea = leads_anon_DF_limpio.filter(regex='_parea', axis=1)
pppparea = leads_anon_DF_limpio.filter(regex='_pppparea', axis=1)
qdur = leads_anon_DF_limpio.filter(regex='_qdur', axis=1)
sdur = leads_anon_DF_limpio.filter(regex='_sdur', axis=1)
tarea = leads_anon_DF_limpio.filter(regex='_tarea', axis=1)
tptparea = leads_anon_DF_limpio.filter(regex='(_tptparea$)', axis=1)
tptpdur = leads_anon_DF_limpio.filter(regex='(_tptpdur$)', axis=1)
st = leads_anon_DF_limpio.filter(regex='(_stend$|_st80$|_ston$)', axis=1)
stslope = leads_anon_DF_limpio.filter(regex='(_stslope)', axis=1)
stdur = leads_anon_DF_limpio.filter(regex='(_stdur$)', axis=1).drop(columns=['V2_stdur'])

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=parea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pppparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=qdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=sdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tarea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=st.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stslope.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stdur.columns)

# CARGA DE DATOS FINALIZADA

# División

### Hipopotasemia

In [54]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 2]
y = x['categoria'].astype(int)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

### Hiperpotasemia

In [496]:
x = leads_anon_DF_limpio[leads_anon_DF_limpio['categoria'] != 1]
y = x['categoria'].astype(int)
y = y.apply(lambda x: 1 if x == 2 else 0)

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

### Regresión

In [510]:
x = leads_anon_DF_limpio
x = x.sample(frac=1, random_state=1)
y = x['K']

x = x.drop(['K', 'categoria'], axis = 1)
x = x.where(pd.notna(x), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)

x = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)

esc = StandardScaler()
x = pd.DataFrame(esc.fit_transform(x), columns=x.columns)

---

# Lasso

### Hipopotasemia

In [481]:
param_grid = {'C': [0.025, 0.05, 0.075, 0.1], 
    'solver': ['liblinear', 'saga']}

modelo = LogisticRegression(class_weight='balanced', max_iter=5000, penalty='l1')
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=5)
grid_search.fit(x, y)

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros hipopotasemia: {'C': 0.1, 'solver': 'liblinear'}
Resultados CV: 0.6897610549117958


In [482]:
param_grid = {'C': [0.1, 0.25, 0.5, 0.75], 
    'solver': ['liblinear', 'saga']}

modelo = LogisticRegression(class_weight='balanced', max_iter=5000, penalty='l1')
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=5)
grid_search.fit(x, y)

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros hipopotasemia: {'C': 0.1, 'solver': 'liblinear'}
Resultados CV: 0.6897440220383502


In [483]:
modelo = LogisticRegression(C=0.1, solver='liblinear', penalty='l1', class_weight='balanced', max_iter=5000)
modelo.fit(x, y)

coeficientes = modelo.coef_[0] != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (77): Index(['aVR_pamp', 'aVL_pamp', 'aVF_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp',
       'V6_pamp', 'III_qamp', 'V1_qamp', 'V2_qamp', 'V6_qamp', 'II_ramp',
       'aVL_ramp', 'V1_ramp', 'V2_ramp', 'V4_ramp', 'II_rdur', 'aVR_rdur',
       'aVL_rdur', 'V2_rdur', 'V3_rdur', 'V5_rdur', 'V6_rdur', 'I_samp',
       'aVR_samp', 'V1_samp', 'V3_samp', 'I_vat', 'II_vat', 'III_vat',
       'aVR_vat', 'aVL_vat', 'aVF_vat', 'V1_vat', 'V2_vat', 'V3_vat', 'V5_vat',
       'III_qrsppk', 'aVR_qrsppk', 'aVL_qrsppk', 'V1_qrsppk', 'V2_qrsppk',
       'V3_qrsppk', 'V4_qrsppk', 'V6_qrsppk', 'II_qrsdur', 'III_qrsdur',
       'aVL_qrsdur', 'aVF_qrsdur', 'V1_qrsdur', 'V1_qrsarea', 'V2_qrsarea',
       'aVR_stmid', 'V1_stmid', 'V2_stmid', 'V3_stmid', 'V6_stmid', 'V2_stdur',
       'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V4_tamp', 'V5_tamp', 'I_tdur',
       'II_tdur', 'aVR_tdur', 'aVF_tdur', 'V1_tdur', 'V3_tdur', 'V6_tdur',
       'II_qtint', 'III_qtint', 'aVL_qtint', 'aVF_qtint', 'V2_q

### Hiperpotasemia

In [500]:
param_grid = {'C': [0.025, 0.05, 0.075, 0.1], 
    'solver': ['liblinear', 'saga']}

modelo = LogisticRegression(class_weight='balanced', max_iter=5000, penalty='l1')
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=5)
grid_search.fit(x, y)

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros hipopotasemia: {'C': 0.1, 'solver': 'liblinear'}
Resultados CV: 0.6307423471889492


In [501]:
param_grid = {'C': [0.1, 0.25, 0.5, 0.75], 
    'solver': ['liblinear', 'saga']}

modelo = LogisticRegression(class_weight='balanced', max_iter=5000, penalty='l1')
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=5)
grid_search.fit(x, y)

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros hipopotasemia: {'C': 0.1, 'solver': 'liblinear'}
Resultados CV: 0.6307164572213116


In [502]:
modelo = LogisticRegression(C=0.1, solver='liblinear', penalty='l1', class_weight='balanced', max_iter=5000)
modelo.fit(x, y)

coeficientes = modelo.coef_[0] != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (68): Index(['II_pamp', 'III_pamp', 'aVR_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp',
       'V5_pamp', 'aVF_qamp', 'V1_qamp', 'V2_qamp', 'V3_qamp', 'V5_qamp',
       'III_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'V2_rdur', 'V3_rdur',
       'V4_rdur', 'II_samp', 'III_samp', 'aVR_samp', 'aVF_samp', 'V1_samp',
       'V4_samp', 'V5_samp', 'V6_samp', 'I_vat', 'aVL_vat', 'V3_vat', 'V4_vat',
       'V5_vat', 'aVR_qrsppk', 'V1_qrsppk', 'V3_qrsppk', 'V4_qrsppk',
       'V6_qrsppk', 'II_qrsdur', 'V1_qrsdur', 'V2_qrsdur', 'V4_qrsdur',
       'V5_qrsdur', 'III_qrsarea', 'aVR_qrsarea', 'aVL_qrsarea', 'V1_qrsarea',
       'V2_qrsarea', 'V3_qrsarea', 'V5_qrsarea', 'V6_qrsarea', 'V2_stmid',
       'V3_stmid', 'V4_stmid', 'V5_stmid', 'V2_stdur', 'II_tamp', 'III_tamp',
       'aVR_tamp', 'V2_tamp', 'V4_tamp', 'V5_tamp', 'aVR_tdur', 'V4_tdur',
       'V5_tdur', 'V2_qtint', 'V3_qtint', 'V6_qtint', 'Edad'],
      dtype='object')


### Regresión

In [515]:
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1]}

modelo = linear_model.Lasso()
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=3)
grid_search.fit(x, y)

print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Mejores parámetros: {'alpha': 0.01}
Resultados CV: 0.030420235765660886


In [516]:
modelo = linear_model.Lasso(alpha=0.01)
modelo.fit(x, y)

coeficientes = modelo.coef_ != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (62): Index(['II_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp', 'V5_pamp',
       'V2_qamp', 'V3_qamp', 'V4_qamp', 'V5_qamp', 'I_ramp', 'aVF_ramp',
       'V1_ramp', 'V6_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'aVF_rdur',
       'V2_rdur', 'V3_rdur', 'II_samp', 'aVL_samp', 'V1_samp', 'V4_samp',
       'V5_samp', 'I_vat', 'aVR_vat', 'V3_vat', 'V4_vat', 'V5_vat',
       'aVR_qrsppk', 'V1_qrsppk', 'V3_qrsppk', 'III_qrsdur', 'V1_qrsdur',
       'I_qrsarea', 'III_qrsarea', 'V1_qrsarea', 'V6_qrsarea', 'II_stmid',
       'V2_stmid', 'V4_stmid', 'V2_stdur', 'II_tamp', 'III_tamp', 'aVR_tamp',
       'V1_tamp', 'V2_tamp', 'I_tdur', 'III_tdur', 'aVF_tdur', 'V1_tdur',
       'V3_tdur', 'V4_tdur', 'V5_tdur', 'V6_tdur', 'aVR_qtint', 'V2_qtint',
       'V3_qtint', 'V6_qtint', 'Sexo', 'Edad'],
      dtype='object')


---

# ElasticNet

### Hipopotasemia

In [484]:
param_grid = {'C': [0.025, 0.05, 0.075, 0.1], 
    'l1_ratio': [0.1, 0.25, 0.5, 0.75]}

modelo = LogisticRegression(solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search.fit(x, (y))

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Mejores parámetros hipopotasemia: {'C': 0.025, 'l1_ratio': 0.1}
Resultados CV: 0.6947066878211487


In [485]:
param_grid = {'C': [0.0001, 0.001, 0.01], 
    'l1_ratio': [0.1, 0.25, 0.5, 0.75]}

modelo = LogisticRegression(solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search.fit(x, (y))

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejores parámetros hipopotasemia: {'C': 0.01, 'l1_ratio': 0.1}
Resultados CV: 0.6895922178242057


In [486]:
param_grid = {'C': [0.0001, 0.001, 0.01], 
    'l1_ratio': [0.0001, 0.001, 0.01]}

modelo = LogisticRegression(solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search.fit(x, (y))

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Mejores parámetros hipopotasemia: {'C': 0.01, 'l1_ratio': 0.01}
Resultados CV: 0.6960985632421502


In [58]:
modelo = LogisticRegression(C=0.01, l1_ratio=0.01, solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
modelo.fit(x, y)

coeficientes = modelo.coef_[0] != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (144): Index(['II_pamp', 'III_pamp', 'aVR_pamp', 'aVL_pamp', 'aVF_pamp', 'V2_pamp',
       'V3_pamp', 'V4_pamp', 'V5_pamp', 'V6_pamp',
       ...
       'aVR_qtint', 'aVL_qtint', 'aVF_qtint', 'V2_qtint', 'V3_qtint',
       'V4_qtint', 'V5_qtint', 'V6_qtint', 'Sexo', 'Edad'],
      dtype='object', length=144)


In [62]:
print(list(car_seleccionadas))

['II_pamp', 'III_pamp', 'aVR_pamp', 'aVL_pamp', 'aVF_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp', 'V5_pamp', 'V6_pamp', 'II_qamp', 'III_qamp', 'aVR_qamp', 'aVL_qamp', 'V1_qamp', 'V2_qamp', 'V3_qamp', 'V4_qamp', 'V6_qamp', 'I_ramp', 'II_ramp', 'III_ramp', 'aVL_ramp', 'aVF_ramp', 'V1_ramp', 'V2_ramp', 'V3_ramp', 'V4_ramp', 'V5_ramp', 'V6_ramp', 'II_rdur', 'III_rdur', 'aVR_rdur', 'aVL_rdur', 'aVF_rdur', 'V1_rdur', 'V2_rdur', 'V3_rdur', 'V4_rdur', 'V5_rdur', 'V6_rdur', 'I_samp', 'II_samp', 'III_samp', 'aVR_samp', 'aVL_samp', 'aVF_samp', 'V1_samp', 'V2_samp', 'V3_samp', 'V4_samp', 'V5_samp', 'V6_samp', 'I_vat', 'II_vat', 'III_vat', 'aVR_vat', 'aVL_vat', 'aVF_vat', 'V1_vat', 'V2_vat', 'V3_vat', 'V4_vat', 'V5_vat', 'I_qrsppk', 'II_qrsppk', 'III_qrsppk', 'aVR_qrsppk', 'aVL_qrsppk', 'V1_qrsppk', 'V2_qrsppk', 'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V6_qrsppk', 'I_qrsdur', 'II_qrsdur', 'III_qrsdur', 'aVR_qrsdur', 'aVL_qrsdur', 'aVF_qrsdur', 'V1_qrsdur', 'V2_qrsdur', 'V3_qrsdur', 'V4_qrsdur', 'V5_qrsdu

### Hiperpotasemia

In [503]:
param_grid = {'C': [0.025, 0.05, 0.075, 0.1], 
    'l1_ratio': [0.1, 0.25, 0.5, 0.75]}

modelo = LogisticRegression(solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search.fit(x, (y))

print(f'Mejores parámetros hipopotasemia: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Mejores parámetros hipopotasemia: {'C': 0.05, 'l1_ratio': 0.5}
Resultados CV: 0.6308372490508413


In [504]:
modelo = LogisticRegression(C=0.05, l1_ratio=0.5, solver='saga', penalty='elasticnet', class_weight='balanced', max_iter=5000)
modelo.fit(x, y)

coeficientes = modelo.coef_[0] != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (70): Index(['II_pamp', 'III_pamp', 'aVR_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp',
       'V5_pamp', 'III_qamp', 'aVF_qamp', 'V1_qamp', 'V2_qamp', 'V3_qamp',
       'V5_qamp', 'III_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'V2_rdur',
       'V3_rdur', 'V4_rdur', 'II_samp', 'III_samp', 'aVR_samp', 'aVF_samp',
       'V1_samp', 'V4_samp', 'V5_samp', 'V6_samp', 'I_vat', 'aVR_vat',
       'aVL_vat', 'V3_vat', 'V4_vat', 'V5_vat', 'aVR_qrsppk', 'V1_qrsppk',
       'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V6_qrsppk', 'II_qrsdur',
       'V1_qrsdur', 'V2_qrsdur', 'V5_qrsdur', 'III_qrsarea', 'aVR_qrsarea',
       'aVL_qrsarea', 'V1_qrsarea', 'V2_qrsarea', 'V3_qrsarea', 'V5_qrsarea',
       'V6_qrsarea', 'V2_stmid', 'V3_stmid', 'V4_stmid', 'V5_stmid',
       'V2_stdur', 'II_tamp', 'III_tamp', 'aVR_tamp', 'V2_tamp', 'V4_tamp',
       'V5_tamp', 'aVR_tdur', 'V4_tdur', 'V5_tdur', 'V2_qtint', 'V3_qtint',
       'V6_qtint', 'Edad'],
      dtype='object')


### Regresión

In [517]:
param_grid = {'alpha': [0.0001, 0.001, 0.01],
             'l1_ratio': [0.1, 0.25, 0.5, 0.75]}

modelo = linear_model.ElasticNet()
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(x, y)

print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Mejores parámetros: {'alpha': 0.01, 'l1_ratio': 0.75}
Resultados CV: 0.026510494741132273


In [518]:
param_grid = {'alpha': [0.01, 0.025, 0.05, 0.075, 0.1],
             'l1_ratio': [0.1, 0.25, 0.5, 0.75]}

modelo = linear_model.ElasticNet()
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(x, y)

print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Mejores parámetros: {'alpha': 0.1, 'l1_ratio': 0.1}
Resultados CV: 0.03437894635695582


In [519]:
param_grid = {'alpha': [0.0001, 0.001, 0.01],
             'l1_ratio': [0.75, 0.85, 0.95]}

modelo = linear_model.ElasticNet()
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(x, y)

print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Mejores parámetros: {'alpha': 0.01, 'l1_ratio': 0.95}
Resultados CV: 0.029793782179895367


In [520]:
param_grid = {'alpha': [0.01, 0.025, 0.05, 0.075, 0.1],
             'l1_ratio': [0.75, 0.85, 0.95]}

modelo = linear_model.ElasticNet()
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=6, scoring='r2', n_jobs=-1)
grid_search.fit(x, y)

print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Resultados CV: {grid_search.best_score_}')

Mejores parámetros: {'alpha': 0.01, 'l1_ratio': 0.95}
Resultados CV: 0.02796026331296869


In [521]:
modelo = linear_model.ElasticNet(alpha=0.1, l1_ratio=0.1)
modelo.fit(x, y)

coeficientes = modelo.coef_ != 0
car_seleccionadas = x.columns[coeficientes]

print(f'Características seleccionadas ({len(car_seleccionadas)}): {car_seleccionadas}')

Características seleccionadas (63): Index(['II_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp', 'V5_pamp',
       'V2_qamp', 'V3_qamp', 'V4_qamp', 'V5_qamp', 'I_ramp', 'aVF_ramp',
       'V1_ramp', 'V6_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'aVF_rdur',
       'V2_rdur', 'V3_rdur', 'II_samp', 'aVL_samp', 'V1_samp', 'V5_samp',
       'I_vat', 'aVR_vat', 'V3_vat', 'V4_vat', 'V5_vat', 'aVR_qrsppk',
       'aVL_qrsppk', 'V1_qrsppk', 'V3_qrsppk', 'V5_qrsppk', 'III_qrsdur',
       'V1_qrsdur', 'I_qrsarea', 'III_qrsarea', 'aVL_qrsarea', 'V1_qrsarea',
       'V6_qrsarea', 'II_stmid', 'V4_stmid', 'V2_stdur', 'II_tamp', 'III_tamp',
       'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'V5_tamp', 'I_tdur',
       'aVF_tdur', 'V1_tdur', 'V3_tdur', 'V4_tdur', 'V5_tdur', 'V6_tdur',
       'V2_qtint', 'V3_qtint', 'V6_qtint', 'Sexo', 'Edad'],
      dtype='object')


# Selección de variables

In [15]:
iniciales_hipo = ['aVR_pamp', 'aVL_pamp', 'aVF_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp',
       'V6_pamp', 'III_qamp', 'V1_qamp', 'V2_qamp', 'V6_qamp', 'II_ramp',
       'aVL_ramp', 'V1_ramp', 'V2_ramp', 'V4_ramp', 'II_rdur', 'aVR_rdur',
       'aVL_rdur', 'V2_rdur', 'V3_rdur', 'V5_rdur', 'V6_rdur', 'I_samp',
       'aVR_samp', 'V1_samp', 'V3_samp', 'I_vat', 'II_vat', 'III_vat',
       'aVR_vat', 'aVL_vat', 'aVF_vat', 'V1_vat', 'V2_vat', 'V3_vat', 'V5_vat',
       'III_qrsppk', 'aVR_qrsppk', 'aVL_qrsppk', 'V1_qrsppk', 'V2_qrsppk',
       'V3_qrsppk', 'V4_qrsppk', 'V6_qrsppk', 'II_qrsdur', 'III_qrsdur',
       'aVL_qrsdur', 'aVF_qrsdur', 'V1_qrsdur', 'V1_qrsarea', 'V2_qrsarea',
       'aVR_stmid', 'V1_stmid', 'V2_stmid', 'V3_stmid', 'V6_stmid', 'V2_stdur',
       'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V4_tamp', 'V5_tamp', 'I_tdur',
       'II_tdur', 'aVR_tdur', 'aVF_tdur', 'V1_tdur', 'V3_tdur', 'V6_tdur',
       'II_qtint', 'III_qtint', 'aVL_qtint', 'aVF_qtint', 'V2_qtint',
       'V3_qtint', 'Sexo']

x = x[iniciales_hipo]

In [505]:
iniciales_hiper = ['II_pamp', 'III_pamp', 'aVR_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp',
       'V5_pamp', 'III_qamp', 'aVF_qamp', 'V1_qamp', 'V2_qamp', 'V3_qamp',
       'V5_qamp', 'III_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'V2_rdur',
       'V3_rdur', 'V4_rdur', 'II_samp', 'III_samp', 'aVR_samp', 'aVF_samp',
       'V1_samp', 'V4_samp', 'V5_samp', 'V6_samp', 'I_vat', 'aVR_vat',
       'aVL_vat', 'V3_vat', 'V4_vat', 'V5_vat', 'aVR_qrsppk', 'V1_qrsppk',
       'V3_qrsppk', 'V4_qrsppk', 'V5_qrsppk', 'V6_qrsppk', 'II_qrsdur',
       'V1_qrsdur', 'V2_qrsdur', 'V5_qrsdur', 'III_qrsarea', 'aVR_qrsarea',
       'aVL_qrsarea', 'V1_qrsarea', 'V2_qrsarea', 'V3_qrsarea', 'V5_qrsarea',
       'V6_qrsarea', 'V2_stmid', 'V3_stmid', 'V4_stmid', 'V5_stmid',
       'V2_stdur', 'II_tamp', 'III_tamp', 'aVR_tamp', 'V2_tamp', 'V4_tamp',
       'V5_tamp', 'aVR_tdur', 'V4_tdur', 'V5_tdur', 'V2_qtint', 'V3_qtint',
       'V6_qtint', 'Edad']

x = x[iniciales_hiper]

In [522]:
iniciales_reg = ['II_pamp', 'V1_pamp', 'V2_pamp', 'V3_pamp', 'V4_pamp', 'V5_pamp',
       'V2_qamp', 'V3_qamp', 'V4_qamp', 'V5_qamp', 'I_ramp', 'aVF_ramp',
       'V1_ramp', 'V6_ramp', 'I_rdur', 'II_rdur', 'aVL_rdur', 'aVF_rdur',
       'V2_rdur', 'V3_rdur', 'II_samp', 'aVL_samp', 'V1_samp', 'V5_samp',
       'I_vat', 'aVR_vat', 'V3_vat', 'V4_vat', 'V5_vat', 'aVR_qrsppk',
       'aVL_qrsppk', 'V1_qrsppk', 'V3_qrsppk', 'V5_qrsppk', 'III_qrsdur',
       'V1_qrsdur', 'I_qrsarea', 'III_qrsarea', 'aVL_qrsarea', 'V1_qrsarea',
       'V6_qrsarea', 'II_stmid', 'V4_stmid', 'V2_stdur', 'II_tamp', 'III_tamp',
       'aVR_tamp', 'V1_tamp', 'V2_tamp', 'V3_tamp', 'V5_tamp', 'I_tdur',
       'aVF_tdur', 'V1_tdur', 'V3_tdur', 'V4_tdur', 'V5_tdur', 'V6_tdur',
       'V2_qtint', 'V3_qtint', 'V6_qtint', 'Sexo', 'Edad']

x = x[iniciales_reg]

## Backwards con AIC - RL

In [16]:
def aic_log(modelo, x, y):
    prob = modelo.predict_proba(x)[:, 1]
    loglik = np.sum(y * np.log(prob) + (1 - y) * np.log(1 - prob))
    d = x.shape[1]
    AIC = - 2 * loglik + 2*d
    return - AIC

In [523]:
def aic_lin(modelo, x, y):
    n_samples, n_features = x.shape
    y_pred = modelo.predict(x)
    rss = np.sum((y - y_pred) ** 2)
    k = n_features
    aic = n_samples * np.log(rss / n_samples) + 2 * k
    return -aic

### Hipopotasemia

In [494]:
modelo = LogisticRegression(penalty=None, max_iter=5000, class_weight='balanced')
sfs = SequentialFeatureSelector(modelo, k_features='best', forward=False, floating=True, verbose=2, scoring=aic_log, cv=skf, n_jobs=-1)
sfs.fit(x, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  77 out of  77 | elapsed:    1.2s finished

[2024-06-28 14:14:01] Features: 76/1 -- score: -564.4674949017242[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  76 out of  76 | elapsed:    1.1s finished

[2024-06-28 14:14:02] Features: 75/1 -- score: -560.4209413151445[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2024-06-28 14:14:04] Features: 74/1 -- score: -556.5392770165114[Parallel(n_jobs=-1)]: U

SequentialFeatureSelector(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
                          estimator=LogisticRegression(class_weight='balanced',
                                                       max_iter=5000,
                                                       penalty=None),
                          floating=True, forward=False, k_features=(1, 77),
                          n_jobs=-1,
                          scoring=<function aic_log at 0x000002286C04AA20>,
                          verbose=2)

In [495]:
print(sfs.k_feature_names_)
print('Caracteristicas seleccionadas:', len(sfs.k_feature_names_))

cars_seleccionadas = sfs.k_feature_names_
cars_seleccionadas = list(cars_seleccionadas)

('aVL_pamp', 'II_ramp', 'V5_rdur', 'aVL_vat', 'V3_vat', 'aVL_qrsppk', 'V1_qrsppk', 'III_qrsdur', 'V1_qrsdur', 'V3_stmid', 'V6_stmid', 'V2_stdur', 'aVR_tamp', 'V1_tamp', 'V5_tamp', 'I_tdur', 'V6_tdur')
Caracteristicas seleccionadas: 17


### Hiperpotasemia

In [508]:
modelo = LogisticRegression(penalty=None, max_iter=5000, class_weight='balanced')
sfs = SequentialFeatureSelector(modelo, k_features='best', forward=False, floating=True, verbose=2, scoring=aic_log, cv=skf, n_jobs=-1)
sfs.fit(x, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    1.1s finished

[2024-06-28 14:19:38] Features: 69/1 -- score: -742.4969036352325[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  69 out of  69 | elapsed:    1.1s finished

[2024-06-28 14:19:39] Features: 68/1 -- score: -738.6439429900091[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  68 out of  68 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2024-06-28 14:19:41] Features: 67/1 -- score: -735.0447483782593[Parallel(n_jobs=-1)]: U

SequentialFeatureSelector(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
                          estimator=LogisticRegression(class_weight='balanced',
                                                       max_iter=5000,
                                                       penalty=None),
                          floating=True, forward=False, k_features=(1, 70),
                          n_jobs=-1,
                          scoring=<function aic_log at 0x000002286C0CBE20>,
                          verbose=2)

In [509]:
print(sfs.k_feature_names_)
print('Caracteristicas seleccionadas:', len(sfs.k_feature_names_))

cars_seleccionadas = sfs.k_feature_names_
cars_seleccionadas = list(cars_seleccionadas)

('V5_qamp', 'I_rdur', 'V5_samp', 'V5_vat', 'V1_qrsdur', 'aVL_qrsarea', 'V1_qrsarea', 'V3_qrsarea', 'V3_stmid', 'V2_tamp', 'V6_qtint')
Caracteristicas seleccionadas: 11


### Regresión

In [525]:
modelo = LinearRegression()
sfs = SequentialFeatureSelector(modelo, k_features='best', forward=False, floating=True, verbose=2, scoring=aic_lin, cv=5, n_jobs=-1)
sfs.fit(x, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  63 out of  63 | elapsed:    3.1s finished

[2024-06-28 14:23:28] Features: 62/1 -- score: 120.10458029539716[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    0.2s finished

[2024-06-28 14:23:28] Features: 61/1 -- score: 123.52919457702126[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2024-06-28 14:23:29] Features: 60/1 -- score: 126.69933956436569[Parallel(n_jobs=-1)]: U

SequentialFeatureSelector(estimator=LinearRegression(), floating=True,
                          forward=False, k_features=(1, 63), n_jobs=-1,
                          scoring=<function aic_lin at 0x000002286D2A62A0>,
                          verbose=2)

In [526]:
print(sfs.k_feature_names_)
print('Caracteristicas seleccionadas:', len(sfs.k_feature_names_))

cars_seleccionadas = sfs.k_feature_names_
cars_seleccionadas = list(cars_seleccionadas)

('I_rdur', 'I_qrsarea', 'V1_qrsarea', 'V2_stdur', 'V2_tamp', 'V6_qtint')
Caracteristicas seleccionadas: 6


## Boruta 

### Hipopotasemia

In [224]:
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1, oob_score=roc_auc_score)
boruta = BorutaPy(estimator=rf, n_estimators='auto', verbose=2)
boruta.fit(x.values, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	137
Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	15
Rejected: 	137
Iteration: 	10 / 100
Confirmed: 	7
Tentative: 	15
Rejected: 	137
Iteration: 	11 / 100
Confirmed: 	7
Tentative: 	15
Rejected: 	137
Iteration: 	12 / 100
Confirmed: 	8
Tentative: 	14
Rejected: 	137
Iteration: 	13 / 100
Confirmed: 	8
Tentative: 	13
Rejected: 	138
Iteration: 	14 / 100
Confirmed: 	8
Tentative: 	13
Rejected: 	138
Iteration: 	15 / 100
Confirmed: 	8
Tentative: 	13
Rejected: 	138
Iteration: 	16 / 100
Confirmed: 	8
Tentat

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced',
                                          n_estimators=56, n_jobs=-1,
                                          oob_score=<function roc_auc_score at 0x000002284DF24680>,
                                          random_state=RandomState(MT19937) at 0x22847C66940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x22847C66940, verbose=2)

In [225]:
car_sel = boruta.support_
car_sel = x.columns[car_sel].values

print(f'Caraterísticas ({len(car_sel)}): {car_sel}')

Caraterísticas (10): ['aVL_pamp' 'V4_ramp' 'I_stmid' 'aVR_stmid' 'V2_stmid' 'V3_stmid'
 'V2_stdur' 'aVR_tamp' 'V2_tamp' 'V3_tamp']


### Hiperpotasemia

In [405]:
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1, oob_score=roc_auc_score)
boruta = BorutaPy(estimator=rf, n_estimators='auto', verbose=2)
boruta.fit(x.values, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	32
Rejected: 	127
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	28
Rejected: 	127
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	28
Rejected: 	127
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	28
Rejected: 	127
Iteration: 	12 / 100
Confirmed: 	4
Tentative: 	24
Rejected: 	131
Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	24
Rejected: 	131
Iteration: 	14 / 100
Confirmed: 	4
Tentative: 	24
Rejected: 	131
Iteration: 	15 / 100
Confirmed: 	4
Tentative: 	24
Rejected: 	131
Iteration: 	16 / 100
Confirmed: 	4
Tentat

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced',
                                          n_estimators=69, n_jobs=-1,
                                          oob_score=<function roc_auc_score at 0x000002284DF24680>,
                                          random_state=RandomState(MT19937) at 0x22847C66940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x22847C66940, verbose=2)

In [406]:
car_sel = boruta.support_
car_sel = x.columns[car_sel].values

print(f'Caraterísticas ({len(car_sel)}): {car_sel}')

Caraterísticas (18): ['I_ramp' 'V5_samp' 'I_qrsppk' 'II_qrsppk' 'aVR_qrsppk' 'V1_qrsppk'
 'V3_qrsppk' 'V4_qrsppk' 'V5_qrsppk' 'V1_qrsdur' 'V5_qrsarea' 'V4_stmid'
 'V2_tamp' 'V3_tamp' 'aVL_tdur' 'III_qtint' 'V6_qtint' 'Edad']


### Regresión

In [209]:
rf = RandomForestRegressor(n_jobs=-1)
boruta = BorutaPy(estimator=rf, n_estimators='auto', verbose=2)
boruta.fit(x.values, y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	159
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	40
Rejected: 	119
Iteration: 	9 / 100
Confirmed: 	6
Tentative: 	34
Rejected: 	119
Iteration: 	10 / 100
Confirmed: 	6
Tentative: 	34
Rejected: 	119
Iteration: 	11 / 100
Confirmed: 	6
Tentative: 	34
Rejected: 	119
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	25
Rejected: 	124
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	25
Rejected: 	124
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	25
Rejected: 	124
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	25
Rejected: 	124
Iteration: 	16 / 100
Confirmed: 	11
T

BorutaPy(estimator=RandomForestRegressor(n_estimators=76, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x22847C66940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x22847C66940, verbose=2)

In [210]:
car_sel = boruta.support_
car_sel = x.columns[car_sel].values

print(f'Caraterísticas ({len(car_sel)}): {car_sel}')

Caraterísticas (20): ['V1_pamp' 'II_ramp' 'V5_samp' 'III_vat' 'V1_vat' 'I_qrsppk' 'aVF_qrsppk'
 'V1_qrsppk' 'V4_qrsppk' 'V2_qrsarea' 'I_stmid' 'V3_stmid' 'aVR_tamp'
 'V1_tamp' 'V2_tamp' 'V3_tamp' 'I_tdur' 'aVL_tdur' 'V6_tdur' 'Edad']
